In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

traffic_df = pd.read_csv('도로종류별 교통량_(좌표,휴게소,사고건수)추가_최종본.csv')
traffic_df.head()

In [ ]:
def cal_dist(x1, y1, x2, y2, a, b):
    area = abs((x1-a) * (y2-b) - (y1-b) * (x2 - a))
    AB = ((x1-x2)**2 + (y1-y2)**2) **0.5
    distance = area/AB
    return distance

In [ ]:
traffic_df.drop('Unnamed: 0',axis=1,inplace=True)
traffic_df.head()

In [ ]:
traffic_df['startpoint_coor'] = traffic_df['startpoint_coor'].apply(ast.literal_eval)
traffic_df['endpoint_coor'] = traffic_df['endpoint_coor'].apply(ast.literal_eval)

In [ ]:
traffic_df['hwamulchaRest_count'] = 0

In [ ]:
traffic_df.head()

In [ ]:
rest_df = pd.read_csv('휴게소데이터.csv')
rest_df.head()

In [ ]:
rest_df = rest_df[rest_df['truckSaYn'] == 'O']
rest_df.head()

In [ ]:
rest_df.reset_index(inplace=True)
rest_df.drop('index',axis=1,inplace=True)

In [ ]:
rest_df

In [ ]:
rest_df.shape

In [ ]:
rest_df.head()

In [ ]:
#각 고속도로 사고별 최단 거리에 있는 고속도로 구간 탐색 -> 점과 직선사이의 거리 공식을 이용해 카운트
pd.options.display.float_format = '{:.5f}'.format

for i in range(len(rest_df)):
    rest_x = rest_df.iloc[i]['위도']
    rest_y = rest_df.iloc[i]['경도']
    
    min_dist_secName = ''
    min_dist = 9999999
    
    for j in range(len(traffic_df)):    
        sect_sx = traffic_df.iloc[j]['startpoint_coor'][0]
        sect_sy = traffic_df.iloc[j]['startpoint_coor'][1]
        sect_ex = traffic_df.iloc[j]['endpoint_coor'][0]
        sect_ey = traffic_df.iloc[j]['endpoint_coor'][1]
        sectName = traffic_df.iloc[j]['구간']
        
        dist = cal_dist(sect_sx,sect_sy,sect_ex,sect_ey,rest_x,rest_y)
        
        if sect_sx > sect_ex:
            tmp = sect_ex
            sect_ex = sect_sx
            sect_sx = tmp
        if sect_sy > sect_ey:
            tmp = sect_ey
            sect_ey = sect_sy
            sect_sy = tmp
        
        if min_dist > dist and ((sect_sx < rest_x and rest_x < sect_ex) or (sect_sy < rest_y and rest_y < sect_ey)):
            min_dist = dist
            min_dist_secName = sectName

    idx = traffic_df[traffic_df['구간'] == min_dist_secName].index.tolist()[0]
    traffic_df.iloc[idx,18] = traffic_df[traffic_df['구간'] == min_dist_secName]['hwamulchaRest_count'] + 1

In [ ]:
traffic_df.to_csv('도로종류별 교통량_(좌표,휴게소,사고건수,화물차휴게소)추가_최종본.csv')

In [ ]:
traffic_df

In [ ]:
traffic_df['hwamulchaRest_count'].value_counts()